# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones

#### Edición 2024

---


## Trabajo práctico entregable - Parte 2

### Integrantes

- Fernando Camino
- Emanuel Nicolás Herrador
- Juana Rapoport
- Sebastián Stutz
- Juan Ignacio Szurlewicz

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_context('talk')

### Lectura del dataset

Leemos nuestro dataset con las respuestas a la encuesta realizada por SysArmy en 2023, en Argentina.

In [3]:
file_url ='https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2023_processed.csv'
df = pd.read_csv(file_url)

# Mostramos el head para ver si se hizo bien la lectura
df.head()

,work_country,work_province,work_dedication,work_contract_type,salary_monthly_BRUTO,salary_monthly_NETO,salary_in_usd,salary_last_dollar_value,salary_has_bonus,salary_bonus_tied_to,...,profile_studies_level_state,profile_career,Institución educativa,¿Salir o seguir contestando sobre las guardias?,work_on_call_duty,salary_on_call_duty_charge,Aclará el número que ingresaste en el campo anterior,¿Salir o seguir contestando sobre estudios?,profile_age,profile_gender
0,Argentina,Catamarca,Full-Time,Staff (planta permanente),680000.0,520000.0,NaN,NaN,No,No recibo bono,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,Varón Cis
1,Argentina,Catamarca,Full-Time,Staff (planta permanente),650000.0,520000.0,NaN,NaN,No,No recibo bono,...,Completo,Licenciatura en Sistemas de Información,Universidad Siglo 21,Terminar encuesta,NaN,NaN,NaN,NaN,45,Varón Cis
2,Argentina,Catamarca,Full-Time,Staff (planta permanente),125000.0,104000.0,NaN,NaN,No,No recibo bono,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,Mujer Cis
3,Argentina,Chaco,Part-Time,Staff (planta permanente),190000.0,240000.0,NaN,NaN,No,No recibo bono,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,Mujer Cis
4,Argentina,Chaco,Full-Time,Tercerizado (trabajo a través de consultora o ...,250000.0,207000.0,NaN,NaN,No,No recibo bono,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,Varón Cis


### Transformación y limpieza del dataset

Para la realización de los siguientes puntos, consideramos importante realizar una limpieza y la transformación correspondiente a nuestros datos.

Para ello, como queremos trabajar con las columnas del salario y del género, vamos a considerar lo siguiente:

- Vamos a quedarnos con aquellas filas que no tengan valor nulo en las columnas `salary_monthly_NETO` y `profile_gender`.
- Vamos a unificar los valores de la columna `profile_gender` en tres posibilidades: `Mujer`, `Varón` y `Diversidad` para poder trabajar con ellas.
- Vamos a considerar un mínimo de $\$73.029,21$ para el salario neto, dado que este límite se obtiene aplicando las [retenciones al sueldo por ley](https://www.argentina.gob.ar/trabajo/buscastrabajo/conocetusderechos/salario#aportes) ($17\%$) al [Salario Mínimo Vital y Móvil](https://www.argentina.gob.ar/noticias/incremento-del-salario-minimo-vital-y-movil) en Argentina a mediados del 2023 ($\$87.987$)
- Vamos a considerar, una vez hechos todos estos filtros, el $99\%$ de los datos con menor salario neto para evitar los casos extremos superiores (casos excepcionales como el de 45M por ejemplo).

In [10]:
# VARIABLES GLOBALES

## - Columnas importantes a considerar
IMPORTANT_COLUMNS = ['salary_monthly_NETO', 'profile_gender', 'gender']

## - Salario Mínimo Vital y Móvil a mediados del 2023
SMVYM = 87_987

## - Porcentaje de aportes obligatorios en Argentina
SALARY_CONTRIBUTIONS = 0.17

## - Percentil a considerar para salarios netos altos
PERCENTILE = 0.99

Dado esto, realizamos la limpieza mencionada y obtenemos el dataset con el que vamos a trabajar.

In [34]:
def print_rows_erased(df, df_cleaned):
    """
    Imprime la cantidad de filas eliminadas
    """
    print(f"Se eliminaron {df.shape[0] - df_cleaned.shape[0]} filas")

def clean_nan_values(df):
    """
    Limpia los valores nulos en las columnas importantes
    """
    df_r = df.dropna(subset=IMPORTANT_COLUMNS)
    print_rows_erased(df, df_r)
    return df_r

def filter_small_salaries(df):
    """
    Filtra los salarios netos menores al Salario Mínimo Vital y Móvil con los aportes
    """
    df_r = df[df['salary_monthly_NETO'] > SMVYM * (1 - SALARY_CONTRIBUTIONS)]
    print_rows_erased(df, df_r)
    return df_r

def filter_outliers(df):
    """
    Filtra los salarios netos altos
    """
    df_r = df[df['salary_monthly_NETO'] < df['salary_monthly_NETO'].quantile(PERCENTILE)]
    print_rows_erased(df, df_r)
    return df_r

def uniform_gender_values(df):
    """
    Unifica los valores de género en el dataset
    """
    dict_transformation = {
        'Varón Cis': 'Varón',
        'Hombre': 'Varón',
        'Mujer': 'Mujer',
        'Mujer Cis': 'Mujer',
        'Femenino': 'Mujer',
        'mujer': 'Mujer',
        'Mujer':'Mujer',
        'Queer':'Diversidades',
        'Varón Trans':'Diversidades',
        'No binarie':'Diversidades',
        'Mujer Trans':'Diversidades',
        'Fluido':'Diversidades',
        'Bigénero':'Diversidades',
        'Gay':'Diversidades'
        }

    df_r = df.copy()

    # A los que no matcheen le ponemos Otro
    df_r['gender'] = df['profile_gender'].map(dict_transformation).fillna('Otro')
    return df_r

# Hago la limpieza
df2 = clean_nan_values(df)
df2 = filter_small_salaries(df2)
df2 = filter_outliers(df2)
df2 = uniform_gender_values(df2)

# Muestro datos de las columnas para ver si se hizo bien la limpieza
print("\nCOLUMNA DE SALARIO:\n", df2['salary_monthly_NETO'].describe().apply(lambda x: format(x, '.2f')))

print("\nCOLUMNA DE GÉNERO:\n", df2['gender'].value_counts())

Se eliminaron 239 filas
Se eliminaron 367 filas
Se eliminaron 52 filas

COLUMNA DE SALARIO:
 count       5147.00
mean      550744.20
std       486441.61
min        73560.00
25%       282379.00
50%       419408.00
75%       600000.00
max      3423000.00
Name: salary_monthly_NETO, dtype: object

COLUMNA DE GÉNERO:
 gender
Varón           3788
Mujer            886
Otro             390
Diversidades      83
Name: count, dtype: int64


In [15]:
df.profile_gender.unique()

array(['Varón Cis', 'Mujer Cis', 'Hombre', 'Prefiero no decir', 'normal',
       'Varon', 'Agénero', 'Masculino', 'Dolarsexual', '.', 'Queer',
       'Mujer Trans', 'No binarie',
       'BIEN MACHO Y PORONGUDO. QUE CLASE DE OPCIONES DE MIERDA SON ESTAS?',
       'no entiendo por que la hacen tan dificil. que es eso de cis? soy heterosexual.',
       'Hombre..... tipo normal', 'Cis', 'varon, no se que es varon cis',
       'Dejen de preguntar boludeces', 'Normal', 'que pelotudos eh',
       'hombre', 'Helicoptero de ataque apache', 'Varón Trans',
       'Varoo natural', 'Varón', 'Bigénero', 'Mujer', 'Transformer',
       'Fluido', 'masculino', 'Hombre, macho silencioso', 'varón',
       'no compliquemos lo simple', 'Agenero, presentación mujer cis',
       'Me considero perteneciente al genero de los que no se identifican con ningun genero',
       'Macho', 'Porque tantas opciones? Solo hay 2. Soy hombre.',
       'heterosexual', 'Machote', 'Puto', 'Gris', 'Heterosexual',
       'Hombre

In [16]:
df.loc[:,'profile_g'] = df.profile_gender.replace({'Varón Cis': 'Varón cis','Mujer': 'Mujer cis','Mujer Cis': 'Mujer cis','Femenino': 'Mujer cis','mujer': 'Mujer cis','Mujer':'Mujer cis','Queer':'Diversidades','Varón Trans':'Diversidades','No binarie':'Diversidades','Mujer Trans':'Diversidades','Fluido':'Diversidades','Bigénero':'Diversidades','Gay':'Diversidades'}).fillna(False)

In [18]:
alpha = 0.05

In [19]:
is_man = df.profile_g == 'Varón cis'
is_woman = df.profile_g == 'Mujer cis'


In [20]:

groupA = df[(df.salary_monthly_NETO > 10000) & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 10000) & is_woman].salary_monthly_NETO

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB).
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos componentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0=...$

**Estadístico (Pivote)**
  * Identificar el estadístico
  * Escribir qué distribución tiene bajo $H_0$


### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

In [21]:
from statsmodels.stats.power import tt_ind_solve_power

In [22]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)
power= 0.8

In [23]:
tt_ind_solve_power(effect_size=effect_size, alpha=alpha,power=power,  ratio=ratio)

163.1896108369031

## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje, en un archivo PDF.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
  1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
  2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
  3. Puntos clave:
    1. Simpleza de los gráficos.
    2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
    2. Selección de UNA oración sobre la que se hace énfasis.
    3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
  2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
  2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
  3. Puntos clave:
    3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
    4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle.
    5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
    2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
  1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
  2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
  3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
  3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
  4. Puntos clave:
    1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
  3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.
